# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
# import gmaps.datasets

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# CSV from WeatherPy
csv_file = pd.read_csv("../WeatherPy/output_data/cities.csv") 

vacay_df = pd.DataFrame(csv_file)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#  configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# vacay_df.head()

# Use the Lat and Lng as locations 
locations = vacay_df[['Lat', 'Lng']]

# Use Humidity as the weight.
humidity = vacay_df['Humidity (%)'].astype(float)


In [5]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure(g_key)

# # Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)


# # Add layer
fig.add_layer(heat_layer)

# # Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# vacay_df

# use .loc to create df with ideal weather conditions
ideal_df = vacay_df.loc[(vacay_df["Max Temp (F)"] < 80) & (vacay_df["Max Temp (F)"] > 70) & 
                        (vacay_df["Wind Speed (mph)"] < 10) & (vacay_df["Cloudiness (%)"] == 0)]
ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
38,38,ladario,43.43,28.33,72.00,94,0,3.36,BG,1596688633
69,69,port alfred,58.67,66.57,72.79,54,0,5.91,RU,1596688641
86,86,portland,38.50,38.00,75.20,29,0,2.24,TR,1596688645
109,109,saint-philippe,-5.18,-68.90,73.67,62,0,0.63,BR,1596688650
187,187,teya,28.42,30.78,75.61,63,0,9.78,EG,1596688670
223,223,taolanaro,33.35,64.42,74.98,17,0,2.62,AF,1596688679
243,243,coquimbo,43.22,27.92,72.00,94,0,3.36,BG,1596688684
252,252,cabo san lucas,-26.79,-60.44,71.69,38,0,9.53,AR,1596688507
259,259,nome,-20.97,164.70,73.94,58,0,7.94,NC,1596688688
274,274,labutta,41.54,-8.62,72.00,100,0,2.53,PT,1596688695


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into variable named hotel_df
hotel_df = ideal_df

# add empty "Hotel Name" column
hotel_df["Hotel Name"] = ""
# hotel_df.count()
hotel_df


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
38,38,ladario,43.43,28.33,72.00,94,0,3.36,BG,1596688633,
69,69,port alfred,58.67,66.57,72.79,54,0,5.91,RU,1596688641,
86,86,portland,38.50,38.00,75.20,29,0,2.24,TR,1596688645,
109,109,saint-philippe,-5.18,-68.90,73.67,62,0,0.63,BR,1596688650,
187,187,teya,28.42,30.78,75.61,63,0,9.78,EG,1596688670,
223,223,taolanaro,33.35,64.42,74.98,17,0,2.62,AF,1596688679,
243,243,coquimbo,43.22,27.92,72.00,94,0,3.36,BG,1596688684,
252,252,cabo san lucas,-26.79,-60.44,71.69,38,0,9.53,AR,1596688507,
259,259,nome,-20.97,164.70,73.94,58,0,7.94,NC,1596688688,
274,274,labutta,41.54,-8.62,72.00,100,0,2.53,PT,1596688695,


In [8]:
# target_coordinates = 
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

for index, row in hotel_df.iterrows():
    params = {
    "location": f"{row['Lat']}, {row['Lng']}",
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    } 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params).json()
    
#     extract results
    results = response['results']
    
    try:
        print(f"The Hotel to stay at is {results[0]['name']}")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result...skipping")

The Hotel to stay at is Royal Grand Hotel and Spa - All Inclusive


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
The Hotel to stay at is Boutique Hotel Antiques&Gold
The Hotel to stay at is Hotel Orel
The Hotel to stay at is Le gîte du Koniambo
The Hotel to stay at is Quinta de Santa Comba
Missing field/result...skipping
The Hotel to stay at is Four Seasons Hotel Alexandria at San Stefano, Egypt
The Hotel to stay at is The Point Coolum Beach - Ocean View Accommodation
Missing field/result...skipping
The Hotel to stay at is Desert Gat


In [9]:
# drop null values
# hotel_df
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
38,38,ladario,43.43,28.33,72.00,94,0,3.36,BG,1596688633,Royal Grand Hotel and Spa - All Inclusive
243,243,coquimbo,43.22,27.92,72.00,94,0,3.36,BG,1596688684,Boutique Hotel Antiques&Gold
252,252,cabo san lucas,-26.79,-60.44,71.69,38,0,9.53,AR,1596688507,Hotel Orel
259,259,nome,-20.97,164.70,73.94,58,0,7.94,NC,1596688688,Le gîte du Koniambo
274,274,labutta,41.54,-8.62,72.00,100,0,2.53,PT,1596688695,Quinta de Santa Comba
406,406,warqla,31.22,29.96,78.80,78,0,8.05,EG,1596688729,"Four Seasons Hotel Alexandria at San Stefano, ..."
474,474,mao,-26.53,153.10,75.99,68,0,1.99,AU,1596688564,The Point Coolum Beach - Ocean View Accommodation
541,541,baruun-urt,31.61,34.76,75.99,52,0,1.99,IL,1596688763,Desert Gat


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# # Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig.add_layer(hotel_markers)

# # Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…